In [180]:
from pyquery import PyQuery as pq
from app.models import Malt
from app import db

In [2]:
doc = pq('http://hobbybrauer.de/forum/wiki/doku.php/malzuebersicht')

In [128]:
def replace(s):
    replace_list = [
        (',', '.'),
        ('~', ''),
        ('?', '')
    ]
    for char in replace_list:
        s = s.replace(char[0], char[1])
    return s
    
def median(s_val):
    values = [str.strip(s) for s in s_val.split('-')]
    try:
        values = [float(replace(s)) for s in values]
    except ValueError:
        return None
    else:    
        return sum(values) / len(values)

In [170]:
def get_malts_from_hobbybrauer():
    for table in doc('table'):
        rows = doc(table).find('tr')
        for row in rows[1:]:
            cells = doc(row).find('td') 

            name = cells.eq(0).text()
            english_name = cells.eq(1).text()
            ebc = median(cells.eq(2).text())

            if name == '':
                name = english_name

            if ebc is not None:
                yield(Malt(name=name, ebc=ebc))

In [171]:
malts = sorted(list(get_malts_from_hobbybrauer()), key=lambda x: x.name)

In [175]:
db.session.add_all(malts)
db.session.commit()